In [2]:
import pandas as pd



In [3]:
# Fetch data
path = "../"
filename = "mydata_withdates.csv"
mydata = pd.read_csv(path+filename)

mydata[0:2]

/Users/christianschleidt/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,19,20,22,40,45,46,47,48,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,...,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location,crea_dates
0,0,471,35525651,02/18/2017 09:17:00 AM,02/21/2017 12:00:00 PM,DSNY,A - Bronx,Snow,E9 Snow / Icy Sidewalk,Sidewalk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-18 09:17:00
1,1,7515,35172585,01/05/2017 08:21:00 AM,01/06/2017 12:00:00 PM,DSNY,BCC - Manhattan,Snow,15S Re-Plow/Spread/Ice-Snow Cond.,Street,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-05 08:21:00


In [4]:
# Key info about the data

#print mydata.dtypes
#print "\nNo. of rows in dataset:", len(mydata.index)

In [5]:
# Clean data - missing observations in crucial columns (location, date, description and type)
missingGeo = mydata[mydata['Latitude'].isnull()]
print "Missing geodata", len(missingGeo)
mydata = mydata.drop(missingGeo.index)

missingGeo = mydata[mydata['Longitude'].isnull()]
print "Missing geodata", len(missingGeo)
mydata = mydata.drop(missingGeo.index)

missingDate = mydata[mydata['Created Date'].isnull()]
print "Missing date", len(missingDate)
mydata = mydata.drop(missingDate.index)

missingDesc = mydata[mydata['Descriptor'].isnull()]
print "Missing descriptor", len(missingDesc)
mydata = mydata.drop(missingDesc.index)

missingType = mydata[mydata['Complaint Type'].isnull()]
print "Missing type", len(missingType)
mydata = mydata.drop(missingType.index)


Missing geodata 28462
Missing geodata 0
Missing date 0
Missing descriptor 30850
Missing type 0


In [8]:
# Filter on data we need for visualization
party = mydata[(mydata.Descriptor == "Loud Music/Party")]
urine = mydata[(mydata["Complaint Type"] == "Urinating in Public")]
rodent = mydata[(mydata["Complaint Type"] == "Rodent")]
graffiti = mydata[(mydata["Complaint Type"] == "Graffiti")]
air = mydata[(mydata["Complaint Type"] == "Air Quality")]

print "Done filtering"

#mydata['crea_dates'] = pd.to_datetime(mydata['Created Date'])
#print "Created date column"

# Comncatenate in new var
vizdata = pd.concat([party, urine, rodent, graffiti, air], axis=0)

print "Done concatenating"

vizdata.reset_index(drop=True, inplace=True)

print "Done resetting index"

print vizdata.dtypes

Done filtering
Done concatenating
Done resetting index
Unnamed: 0                          int64
Unnamed: 0.1                        int64
Unique Key                          int64
Created Date                       object
Closed Date                        object
Agency                             object
Agency Name                        object
Complaint Type                     object
Descriptor                         object
Location Type                      object
Incident Zip                      float64
Incident Address                   object
Street Name                        object
Cross Street 1                     object
Cross Street 2                     object
Intersection Street 1              object
Intersection Street 2              object
Address Type                       object
City                               object
Landmark                           object
Facility Type                      object
Status                             object
Due Date             

In [12]:

# Create/transform dates need to be tranformed to date format
vizdata['crea_dates'] = pd.to_datetime(vizdata['Created Date'])

print "Done creating date column"

vizdata["Year"] = vizdata['crea_dates'].dt.year
#mydata["Weekday"] = mydata['crea_dates'].dt.weekday
#mydata["Hour"] = mydata['crea_dates'].dt.hour
#mydata["Weekday_name"] = mydata['crea_dates'].dt.weekday_name

print "Done creating year column"

#mydata = mydata.assign(
#    TimeOfDay=pd.cut(
#        mydata.Hour,
#        [-1, 5, 11, 17, 24],
#        labels=['Night', 'Morning', 'Afternoon', 'Evening']))

# Remove data from 2017 - year is incomplete
incompleteYear = vizdata[(vizdata["Year"] >= 2017)]
print "Data from 2017", len(missingType)
vizdata = vizdata.drop(incompleteYear.index)

Done creating date column
Done creating year column
Data from 2017 0


In [15]:
# Create category column - used for visualizations
vizdata.loc[(vizdata.Descriptor == "Loud Music/Party"), 'CategoryVis'] = 0
vizdata.loc[(vizdata["Complaint Type"] == "Urinating in Public"), 'CategoryVis'] = 1
vizdata.loc[(vizdata["Complaint Type"] == "Rodent"), 'CategoryVis'] = 2
vizdata.loc[(vizdata["Complaint Type"] == "Graffiti"), 'CategoryVis'] = 3
vizdata.loc[(vizdata["Complaint Type"] == "Air Quality"), 'CategoryVis'] = 3



In [26]:
# Export data to file
exportData = vizdata[["CategoryVis", "Year", "Latitude", "Longitude"]]
exportData.to_csv("viz.csv", sep=',', index=False)